Imports

In [1]:
import tensorflow as tf
import tensorflow as tf
import keras
from keras.layers import Conv2D,MaxPooling2D,Dropout
import os
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split

U- NetModel Creation

In [2]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def UNet(image_size):
    f = [16, 32, 64, 128, 256]
    
    inputs = keras.layers.Input((image_size, image_size, 1))
    print(inputs.shape)
    p0 = inputs
    c1, p1 = down_block(p0, f[0])
    c2, p2 = down_block(p1, f[1]) 
    c3, p3 = down_block(p2, f[2]) 
    c4, p4 = down_block(p3, f[3]) 
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) 
    u2 = up_block(u1, c3, f[2]) 
    u3 = up_block(u2, c2, f[1]) 
    u4 = up_block(u3, c1, f[0])
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [2]:
tf.config.set_visible_devices([], 'GPU')

In [3]:
all_physical_devices = tf.config.list_physical_devices()
all_physical_devices

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Data Preprocessing on Brain Dataset

In [4]:
dataset_path = "Brain_Dataset"
images_path = "Brain_Dataset/data/images"
masks_path = "Brain_Dataset/data/masks"
target_shape = (256,256)
def preprocess_dataset(images_path,masks_path):
    images = []
    masks = []

    for image_file in os.listdir(images_path):
        image_path = os.path.join(images_path, image_file)
        if os.path.isfile(image_path):
            image = cv2.imread(image_path, 0) 
            image = cv2.resize(image, target_shape) 
            images.append(image)

    for mask_file in os.listdir(masks_path):
        mask_path = os.path.join(masks_path, mask_file)
        if os.path.isfile(mask_path):
            mask = cv2.imread(mask_path, 0) 
            mask = cv2.resize(mask, target_shape) 
            masks.append(mask)

    images = np.array(images)
    masks = np.array(masks)

    images = images/255.0
    masks = masks/255.0

    return images,masks

images,masks = preprocess_dataset(images_path,masks_path)

In [62]:
masks[0][128][128]

0.0

Training , Validation and Testing Split of the data

In [5]:
images_train, images_test, masks_train, masks_test = train_test_split(images, masks, test_size= 0.1, random_state=42)
images_train, images_valid, masks_train, masks_valid = train_test_split(images_train, masks_train, test_size= 0.222, random_state=42)

In [75]:
model = UNet(256)
model.summary()

(None, 256, 256, 1)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 256, 256, 16  160         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_20 (Conv2D)             (None, 256, 256, 16  2320        ['conv2d_19[0][0]']              
                                )                                       

Now that our data is prepared we train the model on U-Net for segmentation task

In [77]:
epochs = 10
batch_size = 64

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

history = model.fit(images_train, masks_train, validation_data=(images_valid, masks_valid), epochs=epochs, batch_size=batch_size)

Epoch 1/10
 9/34 [======>.......................] - ETA: 7:50 - loss: 0.1995 - acc: 0.9826

KeyboardInterrupt: 

In [11]:
model.save("brain_mask_model.h5")

Testing the model

In [13]:
predictions = model.predict(images_test)

10/10 [==============================] - 12s 1s/step


In [14]:
loss, accuracy = model.evaluate(images_test, masks_test)
loss,accuracy

10/10 [==============================] - 13s 1s/step - loss: 0.2616 - acc: 0.9825


(0.26158350706100464, 0.9824580550193787)

In [ ]:
masks_train

In [70]:
for i in range(0,245):
    for j in range(0,245):
        if masks[1][i][j] == 1:
            print(i,j)
        


126 128
126 129
126 130
126 131
127 126
127 127
127 128
127 129
127 130
127 131
127 132
127 133
127 134
127 135
127 136
128 124
128 125
128 126
128 127
128 128
128 129
128 130
128 131
128 132
128 133
128 134
128 135
128 136
128 137
128 138
128 139
128 140
129 124
129 125
129 126
129 127
129 128
129 129
129 130
129 131
129 132
129 133
129 134
129 135
129 136
129 137
129 138
129 139
129 140
129 141
130 123
130 124
130 125
130 126
130 127
130 128
130 129
130 130
130 131
130 132
130 133
130 134
130 135
130 136
130 137
130 138
130 139
130 140
130 141
130 142
131 122
131 123
131 124
131 125
131 126
131 127
131 128
131 129
131 130
131 131
131 132
131 133
131 134
131 135
131 136
131 137
131 138
131 139
131 140
131 141
131 142
131 143
131 144
132 121
132 122
132 123
132 124
132 125
132 126
132 127
132 128
132 129
132 130
132 131
132 132
132 133
132 134
132 135
132 136
132 137
132 138
132 139
132 140
132 141
132 142
132 143
132 144
132 145
133 120
133 121
133 122
133 123
133 124
133 125
133 126


In [58]:

#predictions = model.predict(images_test)
image = images[1]
#image = (image * 255).astype(np.uint8)

# Check if the image was loaded successfully
if image is not None:
    # Display the image in a window
    cv2.imshow("Image", image)
    # Wait for a key press and close the window when any key is pressed
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Failed to load image")

In [20]:
image = predictions[3]

# Check if the image was loaded successfully
if image is not None:
    # Display the image in a window
    cv2.imshow("Image", image)
    # Wait for a key press and close the window when any key is pressed
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Failed to load image")